In [566]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self, match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.batting_first = self.get_batting_first()
        self.batting_second = self.get_batting_second()
        # self.team = self.get_team()
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.team1_dismissals_df = self.get_team1_dismissals_df()
        self.team2_dismissals_df = self.get_team2_dismissals_df()
        self.dream11_points = self.get_dream11_points()
        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    # def get_team(self, team):
    #     team = []
    #     for i in self.json_api['rosters'][0]['roster']:
    #          team.append(i['athlete']['name'])
    #     return team
    
    def get_batting_first(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return teamName
            else:
                return otherTeamName[0]
        else:
            if(inningsNumber == '2'):
                return teamName
            else:
                return otherTeamName[0]

    def get_batting_second(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return otherTeamName[0]
            else:
                return teamName
        else:
            if(inningsNumber == '2'):
                return otherTeamName[0]
            else:
                return teamName

    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_team2_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 0
        else:
            z = 1
            
        for i in self.json_api['rosters'][0]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    
    def get_team1_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 1
        else:
            z = 0
        for i in self.json_api['rosters'][1]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df



In [567]:
matchNumber = 1426290
# t = Match(1426295)
m = Match(matchNumber)


/tmp/ipykernel_225035/750235958.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_225035/750235958.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_225035/750235958.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [568]:
# x = m.team1_batting_df

# x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
# x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
# x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where(x['Runs'] > 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])

# y = m.team1_bowling_df

# y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
# y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])

# z = m.team2_dismissals_df

# df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])

# x

In [569]:
# z

In [570]:
# x = pd.merge(x,z)
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x.drop(x.columns[[8,9,10,11]], axis=1, inplace=True)
# x

In [571]:
# catcher = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
# catcher

In [572]:
match = Match(matchNumber)

/tmp/ipykernel_225035/750235958.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_225035/750235958.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_225035/750235958.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [573]:
team1batting = match.team1_batting_df
team2batting = match.team2_batting_df
team1dismiss = match.team1_dismissals_df
team2dismiss = match.team2_dismissals_df
team1bowling = match.team1_bowling_df
team2bowling = match.team2_bowling_df

In [574]:
team1batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Wriddhiman Saha,c †Karthik b Mohammed Siraj,1,7,0,0,14.29
1,Shubman Gill,c Vyshak b Mohammed Siraj,2,7,0,0,28.57
2,Sai Sudharsan,c Kohli b Green,6,14,0,0,42.86
3,M Shahrukh Khan,run out (Kohli),37,24,5,1,154.17
4,David Miller,c Maxwell b Sharma,30,20,3,2,150.00
5,Rahul Tewatia,c Vyshak b Yash Dayal,35,21,5,1,166.67
6,Rashid Khan,b Yash Dayal,18,14,2,1,128.57
7,Vijay Shankar,c Mohammed Siraj b Vyshak,10,7,2,0,142.86
8,Manav Suthar,c Swapnil Singh b Vyshak,1,2,0,0,50.00
9,Mohit Sharma,run out (†Karthik/Vyshak),0,1,0,0,0.00


In [575]:
team1dismiss

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Wriddhiman Saha,Wicketkeeper batter,c,Mohammed Siraj,Dinesh Karthik
1,Shubman Gill,Opening batter,c,Mohammed Siraj,Vijaykumar Vyshak
2,Sai Sudharsan,Top-order batter,c,Cameron Green,Virat Kohli
3,M Shahrukh Khan,Batter,run out,None,Virat Kohli
4,David Miller,Middle-order batter,c,Karn Sharma,Glenn Maxwell
5,Rahul Tewatia,Bowling allrounder,c,Yash Dayal,Vijaykumar Vyshak
6,Rashid Khan,Bowling allrounder,0,Yash Dayal,None
7,Vijay Shankar,Allrounder,c,Vijaykumar Vyshak,Mohammed Siraj
8,Manav Suthar,Bowling allrounder,c,Vijaykumar Vyshak,Swapnil Singh
9,Mohit Sharma,Bowler,run out,None,Dinesh Karthik/Vijaykumar Vyshak


In [576]:
team2batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Virat Kohli,c †Saha b Noor Ahmad,42,27,2,4,155.56
1,Faf du Plessis,c Shahrukh Khan b Little,64,23,10,3,278.26
2,Will Jacks,c Shahrukh Khan b Noor Ahmad,1,3,0,0,33.33
3,Rajat Patidar,c Miller b Little,2,3,0,0,66.67
4,Glenn Maxwell,c Miller b Little,4,3,1,0,133.33
5,Cameron Green,c Shahrukh Khan b Little,1,2,0,0,50.00
6,Dinesh Karthik,not out,21,12,3,0,175.00
7,Swapnil Singh,not out,15,9,2,1,166.67


In [577]:
team2dismiss

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Virat Kohli,Top-order batter,c,Noor Ahmad,Wriddhiman Saha
1,Faf du Plessis,Middle-order batter,c,Josh Little,M Shahrukh Khan
2,Will Jacks,Batting allrounder,c,Noor Ahmad,M Shahrukh Khan
3,Rajat Patidar,Top-order batter,c,Josh Little,David Miller
4,Glenn Maxwell,Batting allrounder,c,Josh Little,David Miller
5,Cameron Green,Batting allrounder,c,Josh Little,M Shahrukh Khan
6,Dinesh Karthik,Wicketkeeper batter,not out,None,None
7,Swapnil Singh,Bowler,not out,None,None
8,Karn Sharma,Bowler,0,None,None
9,Mohammed Siraj,Bowler,0,None,None


In [578]:
team1bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Swapnil Singh,1.0,0,1,0,1.00,5,0,0,0,0
1,Mohammed Siraj,4.0,0,29,2,7.25,13,5,0,3,0
3,Yash Dayal,4.0,0,21,2,5.25,11,2,0,0,0
5,Cameron Green,4.0,0,28,1,7.00,11,3,1,0,0
7,Vijaykumar Vyshak,3.3,0,23,2,6.57,10,2,1,1,0
9,Karn Sharma,3.0,0,42,1,14.00,6,5,3,0,0


In [579]:
team2bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Mohit Sharma,2.0,0,32,0,16.00,2,4,2,0,0
1,Josh Little,4.0,0,45,4,11.25,9,6,2,1,0
3,Manav Suthar,2.0,0,26,0,13.00,4,1,3,0,0
4,Noor Ahmad,4.0,0,23,2,5.75,13,4,0,0,0
6,Rashid Khan,1.4,0,25,0,15.00,1,3,1,0,0


In [580]:
def generate_batting_points(x, y):
    # y = pd.merge(x,y)
    x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    # x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    # x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    # x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    # x.drop(x.columns[[8,9,10,11]], inplace=True, axis=1)
    return x

def generate_bowling_points(y):
    y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])
    return y

# def generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowing, team2bowling):
#     team1batting['Fantasy points'] = team1batting['Runs'] + team1batting['Fours'] + team1batting['Sixes']*2


team1batting = generate_batting_points(team1batting, team1dismiss)
team2batting = generate_batting_points(team2batting, team2dismiss)
team1bowling = generate_bowling_points(team1bowling)
team2bowling = generate_bowling_points(team2bowling)

# generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling)

In [581]:
# pd.merge(team1batting , team1dismiss)
team1batting.merge(team2dismiss[['Name','Position']], how = 'inner')

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate,Fantasy Points,Position


In [582]:
team1dismiss

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Wriddhiman Saha,Wicketkeeper batter,c,Mohammed Siraj,Dinesh Karthik
1,Shubman Gill,Opening batter,c,Mohammed Siraj,Vijaykumar Vyshak
2,Sai Sudharsan,Top-order batter,c,Cameron Green,Virat Kohli
3,M Shahrukh Khan,Batter,run out,None,Virat Kohli
4,David Miller,Middle-order batter,c,Karn Sharma,Glenn Maxwell
5,Rahul Tewatia,Bowling allrounder,c,Yash Dayal,Vijaykumar Vyshak
6,Rashid Khan,Bowling allrounder,0,Yash Dayal,None
7,Vijay Shankar,Allrounder,c,Vijaykumar Vyshak,Mohammed Siraj
8,Manav Suthar,Bowling allrounder,c,Vijaykumar Vyshak,Swapnil Singh
9,Mohit Sharma,Bowler,run out,None,Dinesh Karthik/Vijaykumar Vyshak


In [583]:
team1batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate,Fantasy Points
0,Wriddhiman Saha,c †Karthik b Mohammed Siraj,1,7,0,0,14.29,1
1,Shubman Gill,c Vyshak b Mohammed Siraj,2,7,0,0,28.57,2
2,Sai Sudharsan,c Kohli b Green,6,14,0,0,42.86,6
3,M Shahrukh Khan,run out (Kohli),37,24,5,1,154.17,44
4,David Miller,c Maxwell b Sharma,30,20,3,2,150.00,37
5,Rahul Tewatia,c Vyshak b Yash Dayal,35,21,5,1,166.67,42
6,Rashid Khan,b Yash Dayal,18,14,2,1,128.57,22
7,Vijay Shankar,c Mohammed Siraj b Vyshak,10,7,2,0,142.86,12
8,Manav Suthar,c Swapnil Singh b Vyshak,1,2,0,0,50.00,1
9,Mohit Sharma,run out (†Karthik/Vyshak),0,1,0,0,0.00,-2
